In [3]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer, AutoModel
from global_vars import *
import torch
from torch import nn

model_saved = "bert_mlm.pt"

class BertForClassification(nn.Module):

    def __init__(self, dropout=0.5, freeze=False):

        super(BertForClassification, self).__init__()

        self.model = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 16)
        self.relu = nn.ReLU()
        self.freeze = freeze

        #Freeze Last freeze_threshold Layers
        if self.freeze:
            for param in self.model.embeddings.parameters():
                if(param.requires_grad):
                    param.requires_grad = False


            for i in range(len(self.model.encoder.layer)):
                if(i < len(self.model.encoder.layer) + freeze_threshold):
                    for param in self.model.encoder.layer[i].parameters():
                        if(param.requires_grad):
                            param.requires_grad = False

    def forward(self, input_id, mask):

        _, o1 = self.model(input_ids= input_id, attention_mask=mask, return_dict=False)
        o2 = self.dropout(o1)
        o3 = self.linear(o2)
        fo = self.relu(o3)

        return fo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForClassification()
model.load_state_dict(torch.load(model_saved, map_location=torch.device('cpu')))
model.eval();

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
from torch.nn import functional as F
#input_text = "All things in moderation  Sims is indeed a video game, and a good one at that Note a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim"
#input_text = "No Its like a theme for where I live and that is why I know it by heart"
input_text = "mbti test pretty tough answer office friend awesome besides sleep spend time work anywhere else good understand struggle tps report may well retail version usually price low high end insert miss description hobby box believe people use imagination sensory tend resolve quickly use augment memory meanwhile n tend deal uncertainty generally require think input function understand second sentence mean however still say type use imagination jungian introversion lead people good value imagination zero v n connection funny extrovert value imagination point intuitives deal future abstract handle abstract without imagination think still ni si thing fi judge base inner value think enfps touch emotion nope overwhelm thing relate value overload si likewise entjs intjs pretty aware emotion value uriosity relate imagination instead generate output curiosity create desire input openness correlate intelligence would say correlate n control intuition due desire future model synthesis stp probably curious type nickname detective curiosity intellectual conceptually speak mean thing finesite laugh snowboard omg shout top mountain swim yeah dude rain love run rain eat bread cheese try surf metre wave good banter probably posterboy entj personality harshness come fact try take seriously learn low standard people still expect others try improve work common goal however also idealistic vision future try hard either fulfill trim little religious person believe earnestly religion saw intellectual catholic read technology philosophy economics program psychology neuroscience lead reject religion perhaps main difference entps always need purpose love work extremely quick artist always start point get point b order reach point c help get point actually agree least insofar go female isfj theory behind u pair nf may sound completely disregard massive friction cause completely separate mode think ever try argue enfp infp know mean isfj tend le idealistic humanistic pair well robotic view world pure warmth good spiritedness complement u well run ship make big decision logistics provide insight keep u course like trust advisor tend little overly cautious reserve side work well natural recklessness great dynamic naturally dominant male harmonious relationship end logistical reason hope smooth jazz wine say hello mayhem would everybody second half message unfortunately ridiculous e see something job perceive function really perceive function explore option gather information even generate idea decide narrow say data process judge function judge function perceive ni user ni definitely fiers important enfj friend thing rather te judge function decide thing base effectiveness efficiency ti use different measure efficiency te want efficient resus ti want efficient idea process agree te process judge external world external efficiency internal regulator ti external regulator crossover obviously anything process implement ti prefer former te latter sound like believe perceive function f penay take information te user choose perceive significant penay te user act information know hat bite bias show te user actuary inherent bias yes try rule compare estj enfj friend estjs also move quickly te thing way spin te superior function get butthurt truth need watch put butt say ti bad perception ecause get action need ti twiddle thumb whilst maybe bite accurate entire"

map_val = {0: "ISTJ", 1: "ISFJ", 2: "INFJ", 3: "INTJ", 4: "ISTP", 5: "ISFP", 6: "INFP", 7: "INTP", 8: "ESTP", 9: "ESFP", 10: "ENFP", 11: "ENTP", 12: "ESTJ", 13: "ESFJ", 14: "ENFJ", 15: "ENTJ"}


encoding = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

output = model(encoding['input_ids'].squeeze(1), encoding['attention_mask'])

softmax = F.softmax(output, dim = 1)
top_5 = torch.topk(softmax, 1, dim = 1)[1][0]
for i in top_5:
    print(map_val[i.item()])



ENFJ


In [ ]:
import pickle
data = pickle.load(open("new_train.pkl", "rb"))